In [1]:
import numpy as np
from itertools import combinations

In [12]:
def best_response(payoff_matrixs, k, br):
    """
    Given the a submatrix of player's payoff matrix
    return by best response (may not be NE)
    """
    for M in payoff_matrixs:
        # contruct matrix and vector to be solved
        C = np.empty((k+1, k+1))
        C[:-1, :-1] = M
        C[-1, :-1] = 1
        C[:-1, -1] = -1
        C[-1, -1] = 0
        e = np.zeros(k+1)
        e[-1] = 1
        # use try-except block, because LinAlgError may raise
        try:
            br.append(np.linalg.solve(C, e))
        except np.linalg.LinAlgError:
            return False
    return True

In [19]:
def support_Enumeration(A, B):
    NEs = {}
    m, n = A.shape
    M = np.arange(m)
    N = np.arange(n)
    K = np.min([m, n])
    for k in np.arange(1, K+1, 1):
        NEs[k] = []
        for I in combinations(range(m), k):
            for J in combinations(range(n), k):
                AIJ = A[I, :][:, J]
                BJI = B[I, :][:, J].T
                br = []
                if best_response([AIJ, BJI], k, br):
                    s1, s2 = br
                else:
                    continue
                # Now let's check whether the solution satisfies NE
                if (s1[:-1] > 0).all() & (s2[:-1] > 0).all():
                    if (A[np.setdiff1d(M, I), :][:, J] @ s1[:-1] <= s1[-1]).all() \
                         & (B[I, :][:, np.setdiff1d(N, J)].T @ s2[:-1] <= s2[-1]).all():
                        xI = np.zeros(m)
                        xI[np.array(I)] = s2[:-1]
                        yJ = np.zeros(n)
                        yJ[np.array(J)] = s1[:-1]
                        NEs[k].append({'yJ': yJ,
                                       'u': s1[-1],
                                       'xI': xI,
                                       'v': s2[-1]})
                else:
                    continue
    return NEs

In [20]:
a = np.matrix([[3,3],[2,5],[0,6]])
b = np.matrix([[3,2],[2,6],[3,1]])

In [21]:
support_Enumeration(a,b)

{1: [{'u': 3.0,
   'v': 3.0,
   'xI': array([ 1.,  0.,  0.]),
   'yJ': array([ 1.,  0.])}],
 2: [{'u': 3.0,
   'v': 2.7999999999999998,
   'xI': array([ 0.8,  0.2,  0. ]),
   'yJ': array([ 0.66666667,  0.33333333])},
  {'u': 4.0,
   'v': 2.6666666666666665,
   'xI': array([ 0.        ,  0.33333333,  0.66666667]),
   'yJ': array([ 0.33333333,  0.66666667])}]}

In [22]:
A = np.matrix(
    [[    9504,     -660,    19976,   -20526,     1776,    -8976],
     [ -111771,    31680,  -130944,   168124,    -8514,    52764],
     [  397584,  -113850,   451176,  -586476,    29216,  -178761],
     [  171204,   -45936,   208626,  -263076,    14124,   -84436],
     [ 1303104,  -453420,  1227336, -1718376,    72336,  -461736],
     [  737154,  -227040,   774576, -1039236,    48081,  -300036]]
)
B_T = np.matrix(
    [[   72336,  -461736,  1227336, -1718376,  1303104,  -453420],
     [   48081,  -300036,   774576, -1039236,   737154,  -227040],
     [   29216,  -178761,   451176,  -586476,   397584,  -113850],
     [   14124,   -84436,   208626,  -263076,   171204,   -45936],
     [    1776,    -8976,    19976,   -20526,     9504,     -660],
     [   -8514,    52764,  -130944,   168124,  -111771,    31680]]
)

In [23]:
np.sum([len(i) for i in support_Enumeration(A,B_T.T).values()])

75